In [1]:
import pandas as pd
import numpy as np

In [2]:
def na_imputation(series: pd.Series) -> pd.Series:
    """
    Replace NA values with 0.8 * min of the series (excluding NA).
    Equivalent to the na_imputation() function in R.
    """
    min_val = series.min(skipna=True)
    series_filled = series.fillna(min_val * 0.8)
    return series_filled

In [3]:
# ----------------------------------------------------------------------
# 1. Read data
# ----------------------------------------------------------------------
dat = pd.read_csv('data/ProteinMatrix_sampleID_MapEC50_20240229.csv', index_col=0)

/tmp/ipykernel_8945/2703600708.py:4: DtypeWarning: Columns (0,5587,5588,5589,5590,5591,5592,5593,5596,5599,5600,5601,5603,5604,5606,5614) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv('data/ProteinMatrix_sampleID_MapEC50_20240229.csv', index_col=0)


In [4]:
dat.shape

(15002, 5616)

In [5]:
dat.describe()

,Q8WUW1.Q8WUW1.BRK1_HUMAN.BRK1.Protein.BRICK1,Q9NRR4.Q9NRR4.RNC_HUMAN.DROSHA.Ribonuclease.3,Q99747.Q99747.SNAG_HUMAN.NAPG.Gamma.soluble.NSF.attachment.protein,P10398.P10398.ARAF_HUMAN.ARAF.Serine.threonine.protein.kinase.A.Raf,P05198.P05198.IF2A_HUMAN.EIF2S1.Eukaryotic.translation.initiation.factor.2.subunit.1,Q6P1N0.Q6P1N0.C2D1A_HUMAN.CC2D1A.Coiled.coil.and.C2.domain.containing.protein.1A,Q14807.Q14807.KIF22_HUMAN.KIF22.Kinesin.like.protein.KIF22,Q9H269.Q9H269.VPS16_HUMAN.VPS16.Vacuolar.protein.sorting.associated.protein.16.homolog,O00461.O00461.GOLI4_HUMAN.GOLIM4.Golgi.integral.membrane.protein.4,P49257.P49257.LMAN1_HUMAN.LMAN1.Protein.ERGIC.53,...,Anchor_dose,BioRep,Anchor.Conc,Combo.IC50,EC50,Library.Emax,Bliss.Emax,Combo.Emax,Delta.Xmid,Delta.Emax
count,4996.000000,3222.000000,10491.000000,3741.000000,1.455300e+04,5664.000000,3962.000000,2360.000000,2122.00000,1.355500e+04,...,4721.000000,15002.000000,4504.000000,4513.000000,9247.000000,4513.000000,4513.000000,4513.000000,4513.000000,4504.000000
mean,1118.588424,13351.036862,1849.689544,1021.528857,1.246911e+04,1381.609557,1153.797774,617.659409,648.65159,9.059588e+03,...,4.213818,2.306759,1.921994,11.147487,74.767622,0.617745,0.542388,0.574224,-0.467053,-0.023209
std,582.828276,5973.794412,1080.075213,454.855402,2.628785e+04,614.450956,728.280384,821.810751,371.50948,2.100608e+04,...,7.203862,1.435183,2.347662,3.368156,173.429855,0.246185,0.235177,0.238545,2.172040,0.127101
min,11.710300,176.757000,68.707200,86.684000,5.396670e+02,52.328500,24.259500,44.925400,25.21790,5.350080e+02,...,0.000000,1.000000,0.000250,-1.958610,0.015439,0.002892,0.001377,0.001852,-9.274410,-0.733537
25%,709.249250,9587.340000,1073.735000,698.069000,9.842750e+03,969.951000,755.546000,332.604500,367.99550,5.557585e+03,...,0.100000,1.000000,0.500000,8.770930,5.602588,0.422273,0.370103,0.384653,-1.222430,-0.062574
50%,1072.225000,12562.050000,1648.430000,953.820000,1.191550e+04,1306.650000,1065.635000,492.387500,581.73350,7.271590e+03,...,2.000000,2.000000,1.000000,10.929000,17.359510,0.594500,0.531494,0.596075,-0.185974,-0.009008
75%,1437.647500,16479.300000,2485.010000,1273.560000,1.421200e+04,1669.985000,1426.972500,729.392250,870.10400,9.543580e+03,...,4.000000,3.000000,2.500000,13.946200,69.823265,0.872484,0.764402,0.797490,0.477827,0.025247
max,6414.090000,64541.900000,39161.400000,4507.840000,2.201910e+06,6698.110000,23486.200000,25668.700000,2491.85000,2.355840e+06,...,30.000000,18.000000,7.500000,18.081400,3825.835269,0.960079,0.932757,0.941377,9.311160,0.493566


In [6]:
dat.head(5)

,Q8WUW1.Q8WUW1.BRK1_HUMAN.BRK1.Protein.BRICK1,Q9NRR4.Q9NRR4.RNC_HUMAN.DROSHA.Ribonuclease.3,Q99747.Q99747.SNAG_HUMAN.NAPG.Gamma.soluble.NSF.attachment.protein,P10398.P10398.ARAF_HUMAN.ARAF.Serine.threonine.protein.kinase.A.Raf,P05198.P05198.IF2A_HUMAN.EIF2S1.Eukaryotic.translation.initiation.factor.2.subunit.1,Q6P1N0.Q6P1N0.C2D1A_HUMAN.CC2D1A.Coiled.coil.and.C2.domain.containing.protein.1A,Q14807.Q14807.KIF22_HUMAN.KIF22.Kinesin.like.protein.KIF22,Q9H269.Q9H269.VPS16_HUMAN.VPS16.Vacuolar.protein.sorting.associated.protein.16.homolog,O00461.O00461.GOLI4_HUMAN.GOLIM4.Golgi.integral.membrane.protein.4,P49257.P49257.LMAN1_HUMAN.LMAN1.Protein.ERGIC.53,...,Combo.IC50,EC50,Library.Emax,Bliss.Emax,Combo.Emax,Delta.Xmid,Delta.Emax,Synergy,machine,NY_label
Sample_ID,,,,,,,,,,,,,,,,,,,,,
B1422,NaN,NaN,NaN,NaN,4614.75,NaN,527.226,NaN,NaN,7426.15,...,NaN,10.955999,NaN,NaN,NaN,NaN,NaN,NaN,D,Y
B5718,NaN,NaN,2008.570,NaN,5740.72,NaN,NaN,NaN,NaN,8802.92,...,NaN,85.011714,NaN,NaN,NaN,NaN,NaN,NaN,WAF,N
B3832,NaN,NaN,877.276,NaN,5924.43,NaN,NaN,NaN,NaN,6034.68,...,NaN,10.955999,NaN,NaN,NaN,NaN,NaN,NaN,D,Y
B3637,NaN,NaN,312.174,NaN,5228.96,NaN,NaN,NaN,NaN,8712.97,...,NaN,10.955999,NaN,NaN,NaN,NaN,NaN,NaN,D,Y
B532,NaN,NaN,NaN,NaN,4851.38,NaN,NaN,NaN,NaN,11492.50,...,NaN,10.955999,NaN,NaN,NaN,NaN,NaN,NaN,WAE,Y


In [41]:
dat["Anchor_iname"].value_counts(dropna=False)

Anchor_iname
NaN               10432
Vorinostat          655
Cisplatin           649
Gemcitabine         646
5-Fluorouracil      627
Docetaxel           537
Lapatinib           501
Temozolomide        499
Olaparib            456
Name: count, dtype: int64

In [40]:
dat["Library_iname"].value_counts(dropna=False)

Library_iname
NaN             10432
Cisplatin         562
Afatinib          549
Paclitaxel        541
Lapatinib         490
Docetaxel         466
Sorafenib         464
Alpelisib         419
Olaparib          399
Temozolomide      344
Gemcitabine       336
Name: count, dtype: int64

In [39]:
dat["type2"].value_counts(dropna=False)

type2
NaN                   10161
MCF7_#32 #47             36
MDA-MB-436_#20 #64       36
HCC1187_#53 #64          27
BT549_#64 #32            27
                      ...  
HCC1937_#20 #53           2
HCC1937_#47 #53           1
HCC1937_#33 #53           1
HCC1937_#56 #32           1
HCC1937_#69 #53           1
Name: count, Length: 443, dtype: int64

In [38]:
dat["Anchor.Name"].value_counts(dropna=False)

Anchor.Name
NaN    10489
#64      649
#56      646
#33      631
#69      627
#53      537
#47      493
#32      486
#20      444
Name: count, dtype: int64

In [28]:
len(dat["drugIdAB"].unique())

61

In [37]:
dat["Cell.Line.name"].value_counts(dropna=False)

Cell.Line.name
NaN           10489
BT20            350
HCC1806         327
HCC1143         323
MCF7            322
BT549           306
MDA-MB-231      306
MDA-MB-453      304
T47D            297
MDA-MB-436      279
HCC1937         262
HS578T          260
HCC1187         249
HCC70           246
MDA-MB-468      243
HCC1395         232
HCC38           207
Name: count, dtype: int64

In [36]:
dat["BioRep"].value_counts(dropna=False)

BioRep
3.0     4731
1.0     4604
2.0     4456
4.0      392
5.0      323
6.0      309
7.0       41
8.0       38
9.0       37
10.0      12
11.0      12
12.0      12
13.0       6
14.0       6
15.0       6
16.0       6
17.0       6
18.0       5
Name: count, dtype: int64

In [35]:
dat["cult_time"].value_counts(dropna=False)

cult_time
NaN     10161
24.0     1648
48.0     1606
6.0      1587
Name: count, dtype: int64

In [14]:
dat.columns[-30:]

Index(['Anchor_id', 'Library_iname', 'NY', 'pert_iname', 'pert_id',
       'Anchor_iname', 'Library_id', 'cult_time', 'pert_time', 'type',
       'protein_plate', 'Anchor_dose', 'drugNameAB', 'drugIdAB', 'type2',
       'BioRep', 'Cell.Line.name', 'Anchor.Name', 'Anchor.Conc',
       'Library.Name', 'Combo.IC50', 'EC50', 'Library.Emax', 'Bliss.Emax',
       'Combo.Emax', 'Delta.Xmid', 'Delta.Emax', 'Synergy', 'machine',
       'NY_label'],
      dtype='object')

Anchor_id
    Meaning: An identifier (e.g., numeric or alphanumeric) for the “anchor” drug in a drug-combination experiment.
    Context: In combination screens, one drug is often the “anchor” and tested in combination with many other “library” drugs.

Anchor_iname
    Meaning: The textual or short name for the anchor drug, e.g., a common name or abbreviation.
    Context: Pairs with Anchor_id to describe the drug itself.

Anchor.Name (similar usage) and Anchor.Conc
    Meaning: A descriptive name for the anchor drug, and its concentration (dose) used.
    Context: In some datasets, you’ll see “Anchor_iname” (internal name) vs “Anchor.Name” (more descriptive or standardized).

Anchor_dose
    Meaning: The specific dose at which the anchor drug was applied in a given experiment or well.

Library_id
    Meaning: Unique identifier for a “library” drug in a combination screen.
    Context: This drug is tested in combination with the anchor drug.

Library_iname / Library.Name
    Meaning: The textual or short name of the library drug.

Library.Emax
    Meaning: The maximum effect (Emax) observed/estimated for the library drug alone.

cult_time
    Meaning: The duration (in hours or days) that cells were cultured before the drug(s) were applied.
    Context: Sometimes cells are grown for a certain time prior to perturbation to ensure they are in a specific phase or density.

pert_time
    Meaning: The duration (in hours or days) of the perturbation (drug treatment).
    Context: Important for time-course data and response measurements.

pert_iname / pert_id

Meaning: Generic “perturbation” name or identifier; often corresponds to the name of a drug or combination being tested.
Context: If a single drug is used, this might simply be the drug name; for combos, it might store a code or placeholder.

drugNameAB / drugIdAB

Meaning: A combined name or identifier for a two-drug combination (e.g., “DrugA_DrugB”).
Context: Helps track which two drugs are combined in a single well/experiment.

type / type2

Meaning: Classifications of the experiment, e.g., “singleDrug,” “drugCombination,” or “noDrug” controls. Sometimes there are multiple classification schemes (hence “type2”).

BioRep

Meaning: Biological replicate number (e.g., replicate 1, 2, 3).
Context: Indicates repeated independent experiments (different passages or days) as opposed to technical replicates.

Cell.Line.name

Meaning: The name of the cell line used in the experiment.

protein_plate

Meaning: Identifier for the plate on which protein measurements (e.g., proteomics assays) were conducted.
Context: Large screens run multiple plates; each has its own barcodes, controls, etc.

machine

Meaning: The specific instrument or machine used to measure the data (e.g., a particular mass spectrometer or plate reader).
Context: Could be relevant for batch effects or instrument-specific normalization.

NY / NY_label

Meaning: A site or batch label, possibly “NY” means “New York” or a code for a particular lab site.
Context: Sometimes multiple labs or batch labels are used and “NY” flags which batch or lab the data is from.

EC50

Meaning: The half-maximal effective concentration for a single-drug perturbation (the concentration at which half of the maximum effect is observed).
Context: A common metric in dose–response studies.

Combo.IC50

Meaning: The analogous IC50 (or sometimes the half-maximal inhibitory concentration) for a combination of two drugs.

Bliss.Emax, Combo.Emax

Meaning:
    Bliss.Emax: The predicted or reference maximal effect under the Bliss independence model (used to evaluate synergy).
    Combo.Emax: The actual observed maximal effect from the drug combination.
Context: These columns allow synergy analysis by comparing predicted vs. observed effects.

Delta.Xmid, Delta.Emax, Synergy

Meaning:
    Delta.Xmid / Delta.Emax: Differences in curve parameters (e.g., midpoint of the dose–response curve or max effect) compared to single-drug or predicted combo.
    Synergy: A synergy score (there are multiple synergy models: Bliss, Loewe, ZIP, etc.). Higher synergy means the combo outperforms additive expectations.
Context: Helps quantify whether two drugs together have a greater or lesser effect than expected from single-drug data.

Putting It All Together

Anchor_* columns relate to the anchor drug in a drug-combination screen.
Library_* columns relate to the library drug in that screen.
pert_* columns relate to the general perturbation ID, name, or time.
EC50/IC50/Emax/Delta/Bliss columns are standard pharmacological metrics (dose–response and synergy).
type, Cell.Line.name, machine, BioRep, and NY/batch labeling columns all provide context about the experimental setup (which drug type, which cell line, replicate number, which machine, etc.).

In [27]:
dat.loc[:, "type2"].unique()[:100]

array([nan, 'HCC1187_#53 #64', 'HCC1187_#20 #32', 'HCC1187_#64 #58',
       'HCC1187_#64 #32', 'HCC1187_#56 #58', 'HCC1187_#53 #32',
       'HCC1187_#56 #53', 'HCC1187_#33 #47', 'HCC1187_#32 #47',
       'HCC1187_#47 #21', 'HCC1187_#33 #32', 'HCC1187_#20 #64',
       'HCC1187_no nan', 'HCC1187_#20 #8', 'HCC1187_#56 #21',
       'HCC1187_#69 #53', 'HCC1187_#20 #21', 'HCC1187_#33 #8',
       'HCC1187_#69 #58', 'HCC1187_#53 #21', 'MCF7_#33 #53',
       'MCF7_#47 #56', 'MCF7_#20 #32', 'MCF7_#47 #53', 'MCF7_#47 #64',
       'MCF7_#64 #32', 'MCF7_#53 #32', 'MCF7_#33 #56', 'MCF7_#64 #8',
       'MCF7_#33 #58', 'MCF7_#32 #47', 'MCF7_#32 #8', 'MCF7_#33 #64',
       'MCF7_no nan', 'MCF7_#20 #8', 'MCF7_#33 #21', 'MCF7_#56 #21',
       'MCF7_#69 #47', 'MCF7_#69 #53', 'MCF7_#20 #21', 'MCF7_#56 #37',
       'MCF7_#69 #8', 'MCF7_#53 #37', 'MCF7_#47 #37', 'MCF7_#32 #21',
       'MCF7_#56 #32', 'MCF7_#53 #8', 'MCF7_#69 #58',
       'MDA-MB-436_#20 #53', 'MDA-MB-436_#47 #56', 'MDA-MB-436_#53 #64',
     